## Trackingとcnnを組み合わせた動画認識
Trackingとcnnを組み合わせて動画認識の精度向上を図る。



In [0]:
#driveとの連携
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
cd /content/drive/My Drive/School Project/System Design Project/recognize_num/

/content/drive/My Drive/School Project/System Design Project/recognize_num


In [0]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import time
from chainer import Sequential
import chainer.functions as F
import chainer.links as L
import chainer
from google.colab.patches import cv2_imshow
from pathlib import Path
from chainer import Chain, optimizers, Variable, serializers

In [0]:
#動画ファイルの読み込み
cap = cv2.VideoCapture('/content/drive/My Drive/School Project/System Design Project/recognize_num/movie/num.mp4')
print(cap.isOpened())#読み込めたかどうか

True


In [0]:
#フレームを受け取り二値化する
def shapingImage(frame):
    f_frame = frame.astype(np.uint8)#int変換
    bulr_img =cv2.GaussianBlur(f_frame,(15,15), 0)#平滑化
    threshold_value = 100
    gray_img = cv2.cvtColor(bulr_img, cv2.COLOR_RGB2GRAY)#グレースケール
    ret, thresh_img = cv2.threshold(gray_img, threshold_value, 255, cv2.THRESH_BINARY)#二値化
    return thresh_img

In [0]:
#フレームを受け取り矩形領域リストを取り出す
def getRectList(frame):
    thresh_img = shapingImage(frame)
    fc_img,contours, hierarchy = cv2.findContours(thresh_img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) 
    point_list = list()#矩形領域の座標リスト
    for layer in range(5):#layer = 階層の深さ
        for i in range(0, len(contours)):
            if len(contours[i]) > 0:
                if hierarchy[0][i][3] != layer:
                    continue
                rect = contours[i]
                x, y, w, h = cv2.boundingRect(rect)
                point_list.append([x,y,w,h])
        #矩形領域を抽出できたら
        if len(point_list) != 0:
            break
    else:#矩形領域を抽出できなかったら
        return False
    return point_list

In [0]:
#学習データの読み込み
class CNN(Chain):
    def __init__(self):
        super(CNN, self).__init__(
            conv1 = L.Convolution2D(1, 20, 5), # filter 5
            conv2 = L.Convolution2D(20, 50, 5), # filter 5
            l1 = L.Linear(800, 500),
            l2 = L.Linear(500, 500),
            l3 = L.Linear(500, 10, initialW=np.zeros((10, 500), dtype=np.float32))
        )
    def forward(self, x):
        h = F.max_pooling_2d(F.relu(self.conv1(x)), 2)
        h = F.max_pooling_2d(F.relu(self.conv2(h)), 2)
        h = F.relu(self.l1(h))
        h = F.relu(self.l2(h))
        h = self.l3(h)
        return h
loaded_net = CNN()
chainer.serializers.load_npz('num_cnn.net', loaded_net)

In [0]:
#各設定
output_dir = Path("output_renew_fc")
output_dir.mkdir(exist_ok=True)
n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))  # フレーム数を取得する。
n_digits = len(str(n_frames))  

In [0]:
#とりあえず１フレーム読み込む
ret, frame = cap.read()

In [0]:
first_rectlist = getRectList(frame)
print(first_rectlist)

[[89, 330, 77, 117], [439, 325, 45, 89], [292, 321, 59, 97], [367, 181, 60, 95], [505, 170, 47, 87], [193, 167, 75, 102], [56, 21, 35, 87], [246, 17, 79, 84], [426, 7, 49, 83]]


# 状態遷移
##Tracking状態(adding_mode="NO")
 - trackingを行い追跡物の画像を対象のObjectクラスにaddImageTemp()する。
 - 画像データにtrackingによって求められた矩形領域並びに、対応しているobject名を書き込み、ストリーミングを行う。

##Tracking状態(adding_mode="YES")
 - trackingを行い追跡物の画像,フレーム番号、矩形座標を対象のNameLessObjectクラスにaddInfo()する。

##新規金魚追加タイミング
 - adding_modeフラグを"YES"にする

##金魚名付けタイミング(adding_mode="YES")
 - 受け取った座標位置、flame番号が合致するNameLessObjectListを探索し、合致するNamelessObjectと受け取ったNameから新しいObjectクラスを生成する
 - adding_mode = "DONE"
 - 強制的に認識物消失タイミングに遷移

##認識物出現タイミング (adding_mode="NO")
 - CNNに出現物のデータを投げ、判別されたIDから出現したObjectを特定する。
 - trackerを出現物を対象として生成し、そのtrackerと特定したObjectを紐づける。
 - trackerによって切り取られた矩形画像を対応したObjectにaddImageTemp()する

##認識物出現タイミング (adding_mode="YES")
 - 新しいNameLessObjectを生成しそれとTrackerを紐づける


##認識物消失タイミング (adding_mode="YES")
 - 消失したNameLessObjectのTrackerを削除する

##認識物消失タイミング（adding_mode="DONE")
 - すべてのObjectに対してsaveImageTemp()する
 - 現在のCNNを削除し、新しくCNNをObject数に合わせて作成する
 - すべてのObjectに対してgetLearnAll()を行い取得したデータでCNNを学習させる
 - adding_mode="NO"

##認識物消失タイミング(adding_mode="NO")
 - 全オブジェクトに対してgetLearnTemp()を行い取得したデータでCNNを学習させる。
 - 全オブジェクトに対してsaveImageTemp()を行うことで学習した画像をimagelistに移す

# 制約
- 初期状態では金魚は水槽内にいないものとする
- 複数同時に新しい金魚を登録することはできない
- 名前付けは1匹ずつで行い必ず異なる金魚に対して行う。（金魚の数と、本プログラム内の金魚の数がずれたら終わり）
- 認識されていない金魚は金魚追加モードがoffの時は0匹,onの時は1匹となっていなければならない

# クラス
## 1. Objectクラス
### - 概要
認識を行うオブジェクト
### - 属性
 - tracker : トラッカー
 - id : CNNの目標値となるID
 - name : ユーザによって入力される正解
 - imagelist : trackingによって収集された過去の画像群  
 - imagelist_temp : trackingによって収集された出現期間中の画像群
 

### - ふるまい
 - init(NameLessObject,name) : NameLessObjectとnameを受け取ってname,imagelist,を更新する。
 - setName(name) : nameをセットする
 - setStart_frame(start_frame) : 出現時のframe番号をセットする
 - getLearnTemp() : ImageTempとIDを学習用に成形して渡す
 - getLearnAll()：Imagelistを学習用に成形して渡す
 - addImageTemp(image) :imagelist_tempに画像を追加する
 - saveImageTemp()：imagelist_tempをimagelistに移す
 

## 2. NameLessObjectクラス
### - 概要
認識が完了していないオブジェクト（各リストはindexを共通させておく）
### - 属性
 - framelist : 画面に入ったときから出るときまでのframe番号
 - rectlist : 矩形座標リスト
 - imagelist: 矩形画像リスト
 - tracker : トラッカー
### - ふるまい
 - addInfo(frame_no, rect, image) : それぞれのパラメータを追加する
 - addTracker(tracker) : トラッカーを追加する
 - rmTracker() : トラッカーを削除する

# Objectクラス
認識したい物体を表現するクラス。  
自身に対して学習されたCNNを持つ。

In [0]:
class Object():#認識物クラス（今回は0から9)
    def __init__(self,truename):
        self.truename= truename
        

    def __init__(self,rect,firstframe):#rectはtupleで、
        self.rect = rect
        self.tracker = cv2.TrackerKCF_create()
        self.tracker = tracker.init(firstframe,rect)
    

<class 'cv2.TrackerKCF'>
